In [1]:
import numpy as np
import pandas as pd
import os
import random
random.seed(1)

from tqdm import tqdm_notebook as tqdm

import lightgbm as lgb

In [2]:
input_path = '../datasets/'

In [3]:
# log_df = pd.read_csv(os.path.join(input_path, 'carlog.csv'))
meta_df = pd.read_csv(os.path.join(input_path, 'meta.csv'))
test_df = pd.read_csv(os.path.join(input_path, 'test.csv'))
product_master_df = pd.read_csv(os.path.join(input_path, 'product_master.csv'))


# 正解データの作成

In [4]:
is_predict = True

In [5]:
log_df2 = pd.read_csv('../output/cart_log.csv')
log_df2.head()

/Users/imazekishouta/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,session_id,register_number,date,hour,spend_time,display_action_id,user_id,value_1,name_1,kind_1,number_1,value_2,name_2,kind_2,number_2,unit_price,n_items,is_payment,predict
0,0,2003,2019-02-14,4,0,136,bE94Gct4vGiKM7z2Y79cjk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
1,0,2003,2019-02-14,4,32,209,bE94Gct4vGiKM7z2Y79cjk,4522646719765,おかかおにぎり ５９円,商品,1.0,NaN,NaN,NaN,NaN,59.0,1,0,0
2,0,2003,2019-02-14,4,37,209,bE94Gct4vGiKM7z2Y79cjk,4522646695663,梅（種なし）おにぎり５９,商品,1.0,NaN,NaN,NaN,NaN,59.0,1,0,0
3,0,2003,2019-02-14,4,52,209,bE94Gct4vGiKM7z2Y79cjk,4522646716641,紅鮭おにぎり７９円,商品,1.0,NaN,NaN,NaN,NaN,79.0,1,0,0
4,0,2003,2019-02-14,4,59,209,bE94Gct4vGiKM7z2Y79cjk,4522646695670,明太子おにぎり ７９円,商品,1.0,NaN,NaN,NaN,NaN,79.0,1,0,0


## 購入イベントのあったセッションのみに絞る

In [6]:
def get_paid_event(df):
    print('all sessions: ', len(df.session_id.unique()))

    paid_event_ids = list(df[df['is_payment']==1]['session_id'].unique())
    print('num sessions paid: ', len(paid_event_ids))

    return df[df['session_id'].isin(paid_event_ids)]

paid_event_df = get_paid_event(log_df2)

all sessions:  720175
num sessions paid:  618462


## 商品レコードのみに絞る

In [7]:
def get_item_record(df, cols=['session_id', 'value_1', 'n_items', 'predict']):
    return df[df['kind_1']=='商品'][cols]

paid_item_event_df = get_item_record(paid_event_df)
paid_item_event_df.head()

,session_id,value_1,n_items,predict
1,0,4522646719765,1,0
2,0,4522646695663,1,0
3,0,4522646716641,1,0
4,0,4522646695670,1,0
5,0,4522646720440,1,0


## 商品マスタと紐づけてcategory_idの取得

In [8]:
def join_category_id(df, product_master_df):
    df['value_1'] = df['value_1'].astype('int64')
    df = pd.merge(df, product_master_df[['JAN', 'category_id']], left_on='value_1', right_on='JAN', how='left')
    df.category_id = df.category_id.fillna(-1)
    df.category_id = df.category_id.astype('int')
    return df.drop(['value_1', 'JAN'], axis=1)

category_event_df = join_category_id(paid_item_event_df, product_master_df)
category_event_df.head()

,session_id,n_items,predict,category_id
0,0,1,0,617
1,0,1,0,617
2,0,1,0,617
3,0,1,0,617
4,0,1,0,617


## 対象商品のみに絞る

In [9]:
similar_df = pd.read_csv('../output/similar_categories.csv')
target_ids = list(similar_df.category_id.unique())
print(target_ids)
similar_df.head()

[39, 40, 111, 112, 114, 134, 135, 136, 143, 148, 149, 150, 171, 172, 173, 205, 209, 210, 275, 294, 295, 299, 310, 311, 312, 314, 315, 316, 317, 330, 331, 342, 346, 366, 367, 368, 370, 373, 374, 376, 378, 391, 392, 393, 418, 430, 431, 432, 433, 434, 436, 468, 469, 471, 472, 473, 474, 509, 510, 536, 537, 539, 546, 547, 548, 569, 587, 588, 589, 590, 602, 613, 615, 616, 628, 629, 631, 632, 633, 634, 635, 636, 662, 665, 667, 669, 696, 726, 756, 769, 770, 771]


,category_id,similar_110,similar_113,similar_114,similar_134,similar_171,similar_172,similar_173,similar_376,similar_38,similar_435,similar_467,similar_537,similar_539,similar_629,similar_768
0,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,40,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,111,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,112,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,114,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
def get_target_category_record(df, target_ids):
    return df[df['category_id'].isin(target_ids)]

target_category_event_df = get_target_category_record(category_event_df, target_ids)
target_category_event_df.head()

,session_id,n_items,predict,category_id
19,3,1,0,370
22,3,1,0,205
24,3,1,0,472
27,5,1,0,509
28,5,1,0,209


## 予測対象の行のみに絞る

In [11]:
def get_target_category_record(df, is_predict=True):
    if is_predict:
        df = df[df['predict']==0]
    return df.drop(['predict'], axis=1)

target_row_event_df = get_target_category_record(target_category_event_df, is_predict=is_predict)
target_row_event_df.head()

,session_id,n_items,category_id
19,3,1,370
22,3,1,205
24,3,1,472
27,5,1,509
28,5,1,209


## 購入したか集計

In [12]:
def calc_is_payment(df):
    df = df.groupby(['session_id', 'category_id'])[['n_items']].sum().reset_index()
    df.loc[df['n_items'] > 0, 'n_items'] = 1
    df.loc[df['n_items'] < 0, 'n_items'] = 0
    return df
target_item_sum_df = calc_is_payment(target_row_event_df)
target_item_sum_df.head()

,session_id,category_id,n_items
0,3,205,1
1,3,370,1
2,3,472,1
3,5,209,1
4,5,509,1


## Join similar_df

In [13]:
def join_similar_df(df):
    df = pd.merge(df, similar_df, on='category_id', how='left')
    df.drop(['category_id', 'n_items'], axis=1, inplace=True)
    return df

train_payment_Y = join_similar_df(target_item_sum_df)
train_payment_Y.head()
# 集約しなきゃ！！

,session_id,similar_110,similar_113,similar_114,similar_134,similar_171,similar_172,similar_173,similar_376,similar_38,similar_435,similar_467,similar_537,similar_539,similar_629,similar_768
0,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,5,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# IDで集約

In [14]:
train_payment_Y = train_payment_Y.groupby('session_id').sum()
train_payment_Y.columns

Index(['similar_110', 'similar_113', 'similar_114', 'similar_134',
       'similar_171', 'similar_172', 'similar_173', 'similar_376',
       'similar_38', 'similar_435', 'similar_467', 'similar_537',
       'similar_539', 'similar_629', 'similar_768'],
      dtype='object')

## Join meta.csv

In [15]:
def join_meta_df(df, meta_df):
    session_df = meta_df[['session_id']]
    df = pd.merge(session_df, df, on='session_id', how='left')
    df.fillna(0, inplace=True)
    return df

all_sessions_Y = join_meta_df(train_payment_Y, meta_df)
all_sessions_Y.head()

,session_id,similar_110,similar_113,similar_114,similar_134,similar_171,similar_172,similar_173,similar_376,similar_38,similar_435,similar_467,similar_537,similar_539,similar_629,similar_768
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Train Test Split

In [16]:
def train_test_split(df, test_df):
    # test_Yいらない
    test_sessions = list(test_df['session_id'])
    test_Y = df[df['session_id'].isin(test_sessions)]
    train_Y = df[~df['session_id'].isin(test_sessions)]
    assert len(df) == len(test_Y) + len(train_Y)
    return train_Y, test_Y

train_Y, test_Y = train_test_split(all_sessions_Y, test_df)
display(train_Y.head())
display(test_Y.head())

,session_id,similar_110,similar_113,similar_114,similar_134,similar_171,similar_172,similar_173,similar_376,similar_38,similar_435,similar_467,similar_537,similar_539,similar_629,similar_768
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,session_id,similar_110,similar_113,similar_114,similar_134,similar_171,similar_172,similar_173,similar_376,similar_38,similar_435,similar_467,similar_537,similar_539,similar_629,similar_768
663721,663721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
663725,663725,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
663737,663737,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
663745,663745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
663747,663747,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
if is_predict:
    print('is_predict')
    all_Y = pd.concat([train_Y, test_Y], axis=0).reset_index()
    all_Y.to_csv('../output/given_train_similar_category.csv', index=None)
else:
    all_Y = pd.concat([train_Y, test_Y], axis=0).reset_index()
    all_Y.to_csv('../output/all_similar_category.csv', index=None) # 今からパス直すの面倒なので

is_predict
